In [3]:
import cv2
import cv2 as cv
import numpy as np
import string
import os
from copy import deepcopy

In [10]:
def show_image(title,image):
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def invert_image(image):
    return cv.bitwise_not(image)


img = cv.imread('./svg_formated/0.png')

def preprocess_image(img):
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    
    # otsu threshold
    thresh_min = cv.threshold(gray, 1, 255, cv.THRESH_BINARY|cv.THRESH_OTSU)[1]
    
    thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)[1]
    
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3,8))
    morph_img = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)
    
    kernel = np.ones((2, 2), np.uint8)
    morph_img = cv.erode(morph_img, kernel)
    
    kernel = np.ones((4, 4), np.uint8)
    morph_img = cv.dilate(morph_img, kernel)
    
    image1 = morph_img.tolist()
    image2 = thresh_min.tolist()
    
    my_img = [[0 for i in range(len(x))] for x in image1]
    
    for i in range(len(image1)):
        for j in range(len(image1[0])):

            my_img[i][j] = max(0,image1[i][j] - image2[i][j])
    
    
    my_img = np.array(my_img)
    my_img = my_img.astype(np.uint8)
    
    return invert_image(my_img)

image_without_noise = preprocess_image(img)




In [ ]:
show_image("img",image_without_noise)